# Thesis Notebook
## 1 Data Retrieval

In [1]:
import pandas as pd
import numpy as np
import re

## WIPO IPC codes

In [2]:
excel_file_path = 'IPC WIPO unione (1).xlsx'  
# Read the Excel file into a DataFrame
WIPO_IPC = pd.read_excel(excel_file_path)
WIPO_IPC_codes = WIPO_IPC.values.flatten()

# Adding a white space after the fourth character
WIPO_IPC_codes = [code[:4] + '  ' + code[4:] for code in WIPO_IPC_codes]



# Convert each value to a string to avoid issues with non-string data types
WIPO_IPC_code_list = ["'" + code + "'" for code in WIPO_IPC_codes]


In [5]:
WIPO_IPC_code_list

["'B60L  7/00'",
 "'B60L  7/02'",
 "'B60L  7/04'",
 "'B60L  7/06'",
 "'B60L  7/08'",
 "'B60L  7/10'",
 "'B60L  7/12'",
 "'B60L  7/14'",
 "'B60L  7/16'",
 "'B60L  7/18'",
 "'B60L  7/20'",
 "'B60L  7/22'",
 "'B60L  7/24'",
 "'B60L  7/26'",
 "'B60L  7/28'",
 "'B60L  50/00'",
 "'B60L  50/10'",
 "'B60L  50/11'",
 "'B60L  50/12'",
 "'B60L  50/13'",
 "'B60L  50/14'",
 "'B60L  50/15'",
 "'B60L  50/16'",
 "'B60L  50/20'",
 "'B60L  50/30'",
 "'B60L  50/40'",
 "'B60L  50/50'",
 "'B60L  50/51'",
 "'B60L  50/52'",
 "'B60L  50/53'",
 "'B60L  50/60'",
 "'B60L  50/61'",
 "'B60L  50/62'",
 "'B60L  50/64'",
 "'B60L  50/70'",
 "'B60L  50/71'",
 "'B60L  50/72'",
 "'B60L  50/75'",
 "'B60L  50/90'",
 "'B60L  53/00'",
 "'B60L  53/10'",
 "'B60L  53/12'",
 "'B60L  53/122'",
 "'B60L  53/124'",
 "'B60L  53/126'",
 "'B60L  53/14'",
 "'B60L  53/16'",
 "'B60L  53/18'",
 "'B60L  53/20'",
 "'B60L  53/22'",
 "'B60L  53/24'",
 "'B60L  53/30'",
 "'B60L  53/302'",
 "'B60L  53/31'",
 "'B60L  53/34'",
 "'B60L  53/35'",
 "'

In [3]:
# define function to take key words and return a list of SQL queries specifying the tables and fields to search, with each query limited to 10,000 characters (patstat requirement)
def split_codes_into_queries(codes, max_spaces, template_overhead):
    """
    Split conditions into groups such that each group's SQL query remains within the specified character limit.
    """
    # Initialize variables to hold condition groups and current query length
    query_groups = []
    current_group = []
    current_length = template_overhead  # Start with overhead of the fixed part of the SQL template

    for code in codes:
        # Calculate the length of the condition with ' OR ' separator
        code_length = len(code) + 2  # Account for ', '

        if current_length + code_length <= max_spaces:
            # If adding this condition doesn't exceed the limit, add it to the current group
            current_group.append(code)
            current_length += code_length
        else:
            # If it exceeds the limit, save the current group and start a new one
            query_groups.append(current_group)
            # Reset current group with the new condition and the overhead
            current_group = [code]
            current_length = template_overhead + code_length

    # Add the last group if it's not empty
    if current_group:
        query_groups.append(current_group)

    return query_groups


def generate_sql_queries_cpc(codes, type, max_spaces=10000):
    """
    Generate SQL queries based on 'NEAR' patterns, limiting each query to a specified number of spaces.
    """
    # Generate conditions for all given 'NEAR' patterns
    if type == "removespaces":
    # The static part of the SQL query (before inserting conditions)
        sql_template_base = """

    SELECT Distinct PUBLN_NR, PUBLN_KIND, publn_auth, publn_date, cpc_class_symbol
    FROM reg101_appln
    INNER JOIN reg102_pat_publn
    ON reg101_appln.id = reg102_pat_publn.id
    INNER JOIN tls224_appln_cpc
    ON reg101_appln.appln_id = tls224_appln_cpc.appln_id
    WHERE publn_auth = 'EP' 
    AND REPLACE(cpc_class_symbol, ' ', '') IN 
    ( {codes}  )
    """
    elif type == "original":
        sql_template_base = """

    SELECT Distinct PUBLN_NR, PUBLN_KIND, publn_auth, publn_date, cpc_class_symbol
    FROM reg101_appln
    INNER JOIN reg102_pat_publn
    ON reg101_appln.id = reg102_pat_publn.id
    INNER JOIN tls224_appln_cpc
    ON reg101_appln.appln_id = tls224_appln_cpc.appln_id
    WHERE publn_auth = 'EP' 
    AND cpc_class_symbol IN 
    ( {codes}  )
    """

    # Get the length of the static part of the query
    template_overhead = len(sql_template_base.format(codes=''))

    # Split conditions into groups with the given max spaces limit
    query_groups = split_codes_into_queries(codes, max_spaces, template_overhead)

    # List to hold the final SQL queries
    sql_queries = []    

    # Generate a complete SQL query for each group of conditions
    for group in query_groups:
        formatted_codes = ", ".join(group)  # Join with 'OR'
        final_sql = sql_template_base.format(codes=formatted_codes)  # Insert into template
        sql_queries.append(final_sql)  # Add to list of final SQL queries

    return sql_queries


def generate_sql_queries_ipc(codes, type, max_spaces=10000):
    """
    Generate SQL queries based on 'NEAR' patterns, limiting each query to a specified number of spaces.
    """
    # Generate conditions for all given 'NEAR' patterns
    
    if type == "removespaces":
    # The static part of the SQL query (before inserting conditions)
        sql_template_base = """

    SELECT Distinct PUBLN_NR, PUBLN_KIND, publn_auth, publn_date, ipc_class_symbol
    FROM reg101_appln
    INNER JOIN reg102_pat_publn
    ON reg101_appln.id = reg102_pat_publn.id
    INNER JOIN tls209_appln_ipc
    ON reg101_appln.appln_id = tls209_appln_ipc.appln_id
    WHERE publn_auth = 'EP' 
    AND REPLACE(ipc_class_symbol, ' ', '') IN 
    ( {codes}  )
    """
    elif type == "original":
        sql_template_base = """

    SELECT Distinct PUBLN_NR, PUBLN_KIND, publn_auth, publn_date, ipc_class_symbol
    FROM reg101_appln
    INNER JOIN reg102_pat_publn
    ON reg101_appln.id = reg102_pat_publn.id
    INNER JOIN tls209_appln_ipc
    ON reg101_appln.appln_id = tls209_appln_ipc.appln_id
    WHERE publn_auth = 'EP' 
    AND ipc_class_symbol IN 
    ( {codes}  )
    """

    # Get the length of the static part of the query
    template_overhead = len(sql_template_base.format(codes=''))

    # Split conditions into groups with the given max spaces limit
    query_groups = split_codes_into_queries(codes, max_spaces, template_overhead)

    # List to hold the final SQL queries
    sql_queries = []    

    # Generate a complete SQL query for each group of conditions
    for group in query_groups:
        formatted_codes = ", ".join(group)  # Join with 'OR'
        final_sql = sql_template_base.format(codes=formatted_codes)  # Insert into template
        sql_queries.append(final_sql)  # Add to list of final SQL queries

    return sql_queries


In [4]:
WIPO_IPC_queries = generate_sql_queries_ipc(WIPO_IPC_code_list, "original", max_spaces=4000)
for query in WIPO_IPC_queries:
    print(query)
    print("\n")



    SELECT Distinct PUBLN_NR, PUBLN_KIND, publn_auth, publn_date, ipc_class_symbol
    FROM reg101_appln
    INNER JOIN reg102_pat_publn
    ON reg101_appln.id = reg102_pat_publn.id
    INNER JOIN tls209_appln_ipc
    ON reg101_appln.appln_id = tls209_appln_ipc.appln_id
    WHERE publn_auth = 'EP' 
    AND ipc_class_symbol IN 
    ( 'B60L  7/00', 'B60L  7/02', 'B60L  7/04', 'B60L  7/06', 'B60L  7/08', 'B60L  7/10', 'B60L  7/12', 'B60L  7/14', 'B60L  7/16', 'B60L  7/18', 'B60L  7/20', 'B60L  7/22', 'B60L  7/24', 'B60L  7/26', 'B60L  7/28', 'B60L  50/00', 'B60L  50/10', 'B60L  50/11', 'B60L  50/12', 'B60L  50/13', 'B60L  50/14', 'B60L  50/15', 'B60L  50/16', 'B60L  50/20', 'B60L  50/30', 'B60L  50/40', 'B60L  50/50', 'B60L  50/51', 'B60L  50/52', 'B60L  50/53', 'B60L  50/60', 'B60L  50/61', 'B60L  50/62', 'B60L  50/64', 'B60L  50/70', 'B60L  50/71', 'B60L  50/72', 'B60L  50/75', 'B60L  50/90', 'B60L  53/00', 'B60L  53/10', 'B60L  53/12', 'B60L  53/122', 'B60L  53/124', 'B60L  53/126', 

## ENVTECH

In [5]:
file_path = 'ENVTECH_green_codes.csv'  # Change to your file's path if needed

# Read the text file using tab as the delimiter
ENVTECH_codes = pd.read_csv(file_path, header=None)
ENVTECH_codes.columns = ENVTECH_codes.iloc[0]  # Set the zeroth row as column names
# Drop the first row since it's now used for column names
ENVTECH_codes = ENVTECH_codes[1:]
ENVTECH_codes.reset_index(drop=True, inplace=True)
ENVTECH_codes.head()


,Code
0,B01D53/34
1,B01D53/343
2,B01D53/346
3,B01D53/38
4,B01D53/40


In [6]:
ENVTECH_codes = ENVTECH_codes.values.flatten()
# Adding a white space after the fourth character
ENVTECH_codes = [code[:4] + '  ' + code[4:] for code in ENVTECH_codes]

# Convert each value to a string to avoid issues with non-string data types
ENVTECH_code_list = ["'" + code + "'" for code in ENVTECH_codes]

In [7]:
ENVTECH_IPC_queries = generate_sql_queries_ipc(ENVTECH_code_list, "original", max_spaces=10000)
ENVTECH_CPC_queries = generate_sql_queries_cpc(ENVTECH_code_list, "original")
for query in ENVTECH_CPC_queries:
    print(query)
    print("\n")
##for query in ENVTECH_CPC_queries:
#    print(query)
#    print("\n")



    SELECT Distinct PUBLN_NR, PUBLN_KIND, publn_auth, publn_date, cpc_class_symbol
    FROM reg101_appln
    INNER JOIN reg102_pat_publn
    ON reg101_appln.id = reg102_pat_publn.id
    INNER JOIN tls224_appln_cpc
    ON reg101_appln.appln_id = tls224_appln_cpc.appln_id
    WHERE publn_auth = 'EP' 
    AND cpc_class_symbol IN 
    ( 'B01D  53/34', 'B01D  53/343', 'B01D  53/346', 'B01D  53/38', 'B01D  53/40', 'B01D  53/42', 'B01D  53/44', 'B01D  53/46', 'B01D  53/48', 'B01D  53/485', 'B01D  53/50', 'B01D  53/501', 'B01D  53/502', 'B01D  53/504', 'B01D  53/505', 'B01D  53/507', 'B01D  53/508', 'B01D  53/52', 'B01D  53/523', 'B01D  53/526', 'B01D  53/54', 'B01D  53/56', 'B01D  53/565', 'B01D  53/58', 'B01D  53/60', 'B01D  53/62', 'B01D  53/64', 'B01D  53/66', 'B01D  53/68', 'B01D  53/685', 'B01D  53/70', 'B01D  53/72', 'B01D  53/73', 'B01D  53/74', 'B01D  53/75', 'B01D  53/76', 'B01D  53/77', 'B01D  53/78', 'B01D  53/79', 'B01D  53/80', 'B01D  53/81', 'B01D  53/82', 'B01D  53/83', 'B01D

In [8]:
'Y02P  20/582', 'Y02P  20/584', 'Y02P  20/59', 'Y02P  30/00', 'Y02P  30/20', 'Y02P  30/40', 'Y02P  40/00', 'Y02P  40/10', 'Y02P  40/121', 'Y02P  40/125', 'Y02P  40/18', 'Y02P  40/40' , 'Y02P  40/45', 'Y02P  40/50' 

('Y02P  20/582',
 'Y02P  20/584',
 'Y02P  20/59',
 'Y02P  30/00',
 'Y02P  30/20',
 'Y02P  30/40',
 'Y02P  40/00',
 'Y02P  40/10',
 'Y02P  40/121',
 'Y02P  40/125',
 'Y02P  40/18',
 'Y02P  40/40',
 'Y02P  40/45',
 'Y02P  40/50')

In [9]:
csv_file_paths = [
    'wipo1_mit_auth.csv',
    'wipo2_mit_auth.csv',
    'wipo3_mit_auth.csv',
    'wipo4_mit_auth.csv',
    'wipo5_mit_auth.csv',
    'wipo6_mit_auth.csv',
    'wipo7_mit_auth.csv',
    'wipo8_mit_auth.csv'
]

# Initialize an empty list to hold DataFrames
dataframes = []

# Read each CSV file and store the DataFrame in the list
for path in csv_file_paths:
    # Read the CSV file
    df = pd.read_csv(path, sep=';')  # Adjust delimiter as needed
    
    # Select only the relevant columns, if necessary
    df = df[['PUBLN_NR', 'PUBLN_KIND', 'publn_auth', 'publn_date']]  # Adjust column names if different
    
    # Append the DataFrame to the list
    dataframes.append(df)

# Concatenate all DataFrames into one large DataFrame
WIPO_IPC_identifiers = pd.concat(dataframes, ignore_index=True)

# Display the resulting DataFrame
len(WIPO_IPC_identifiers)

80389

In [10]:
file_path = 'cpc-section-Y_20240501.txt'  # Change to your file's path if needed

# Read the text file using tab as the delimiter
CPC_y_codes = pd.read_csv(file_path, sep='\t', header=None)

# Display the first few rows to understand the structure
CPC_y_codes.head()

,0,1,2
0,Y,NaN,GENERAL TAGGING OF NEW TECHNOLOGICAL DEVELOPME...
1,Y02,NaN,TECHNOLOGIES OR APPLICATIONS FOR MITIGATION OR...
2,Y02A,NaN,TECHNOLOGIES FOR ADAPTATION TO CLIMATE CHANGE
3,Y02A10/00,0.0,at coastal zones; at river basins
4,Y02A10/11,1.0,"Hard structures, e.g. dams, dykes or breakwaters"


In [11]:
CPC_y_codes_flat = CPC_y_codes.iloc[:, 0]
#filtered_values = flattened_values[flattened_values.str.contains(r'Y02B|Y02C|Y02D|Y02E|Y02P|Y02T|Y02W|Y04S', case=False, na=False)]
CPC_y_codes_filtered = CPC_y_codes_flat[CPC_y_codes_flat.str.contains(
    r'Y02B|Y02C|Y02D|Y02E|Y02P|Y02T|Y02W|Y04S', case=False, na=False)]

CPC_y_codes_filtered = [code[:4] + '  ' + code[4:] for code in CPC_y_codes_filtered]
CPC_y_code_list = ["'" + code + "'" for code in CPC_y_codes_filtered]


In [12]:
Y_CPC_queries = generate_sql_queries_cpc(CPC_y_code_list, "original")
for query in Y_CPC_queries:
    print(query)
    print("\n")



    SELECT Distinct PUBLN_NR, PUBLN_KIND, publn_auth, publn_date, cpc_class_symbol
    FROM reg101_appln
    INNER JOIN reg102_pat_publn
    ON reg101_appln.id = reg102_pat_publn.id
    INNER JOIN tls224_appln_cpc
    ON reg101_appln.appln_id = tls224_appln_cpc.appln_id
    WHERE publn_auth = 'EP' 
    AND cpc_class_symbol IN 
    ( 'Y02B  ', 'Y02B  10/00', 'Y02B  10/10', 'Y02B  10/20', 'Y02B  10/30', 'Y02B  10/40', 'Y02B  10/50', 'Y02B  10/70', 'Y02B  20/00', 'Y02B  20/30', 'Y02B  20/40', 'Y02B  20/72', 'Y02B  30/00', 'Y02B  30/12', 'Y02B  30/13', 'Y02B  30/17', 'Y02B  30/18', 'Y02B  30/52', 'Y02B  30/54', 'Y02B  30/56', 'Y02B  30/62', 'Y02B  30/625', 'Y02B  30/70', 'Y02B  30/90', 'Y02B  40/00', 'Y02B  40/18', 'Y02B  50/00', 'Y02B  70/00', 'Y02B  70/10', 'Y02B  70/30', 'Y02B  70/3225', 'Y02B  70/34', 'Y02B  80/00', 'Y02B  80/10', 'Y02B  80/22', 'Y02B  80/32', 'Y02B  90/00', 'Y02B  90/10', 'Y02B  90/20', 'Y02C  ', 'Y02C  20/00', 'Y02C  20/10', 'Y02C  20/20', 'Y02C  20/30', 'Y02C  20/